In [1]:
!pip install numpy
!pip install open3d
!pip install ipywidgets
!pip install opencv-python

In [29]:
import numpy as np
import open3d as o3d
import pathlib
import re
from ipywidgets import interact
from lib.datasets.kitti_utils import Calibration
import cv2

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### 读取图像

In [30]:
from tools.dataset_util import Dataset
idx = 13
dataset = Dataset("train", r"/mnt/e/DataSet/kitti")
image, depth = dataset.get_image_with_depth(idx)
calib = dataset.get_calib(idx)
print(image.shape, depth.shape)

(375, 1242, 3) (375, 1242)


#### 图像投影至 rect

In [3]:
h, w = depth.shape
u = np.repeat(np.arange(w), h)
v = np.tile(np.arange(h), w)
d = depth[v, u]
rgb = image[v, u][:, ::-1]
points = calib.img_to_rect(u, v, d)
print(points.shape)

(465750, 3)


#### 获取标签

In [4]:
obj_list = dataset.get_label(idx)

bbox3d = dataset.get_bbox3d_rect(idx, on_button=False)
print(bbox3d)

[[-3.16000009  1.04499996 20.12999916  3.47        1.45        1.56
   1.59      ]]


In [22]:
ry = bbox3d[:, -1]
xyz = bbox3d[:, 0:3]
xyz_ = xyz.copy()
xyz_[:, 0] += 5
xyz_[:, 2] += 5

def get_ry_(xyz, xyz_, ry, calib:Calibration):
    uv_, _ = calib.rect_to_img(xyz_)
    uv, _ = calib.rect_to_img(xyz)
    alpha = calib.ry2alpha(ry, uv[:,0])
    ry_ = calib.alpha2ry(alpha, uv_[:,0])
    return ry_

calib = dataset.get_calib(idx)
ry_ = get_ry_(xyz, xyz_, ry, calib)

print(ry_)

[1.81824249]


#### Data Augment

In [23]:
from tools.box_util import remove_points_in_boxes3d, get_objects_in_boxes3d
points_with_rgb = np.concatenate([points, rgb], axis=1)
enlarge = 0.2
no_object = remove_points_in_boxes3d(points_with_rgb, bbox3d, enlarge)
objects = get_objects_in_boxes3d(points_with_rgb, bbox3d, enlarge)

In [24]:

def translate(points, xyz, ry, xyz_, ry_):
    points = points.copy()
    points -= xyz
    theta = ry_ - ry
    rot_mat = np.array([
        [np.cos(theta), 0, np.sin(theta)],
        [0, 1, 0],
        [-np.sin(theta), 0, np.cos(theta)]
    ])
    points = np.dot(points, rot_mat.T)
    points += xyz_
    return points


for i, obj in enumerate(objects):
    cord = obj[:,0:3]
    obj[:,0:3] = translate(cord, xyz[i], ry[i], xyz_[i], ry_[i])

aug_points = np.concatenate([no_object, *objects], axis=0)


#### 投影回图像

In [25]:
uv, depth = calib.rect_to_img(aug_points[:, 0:3])
u, v = np.round(uv[:, 0]).astype(int), np.round(uv[:, 1]).astype(int)
width, height = u.max() + 1, v.max() + 1

img_no_obj = np.zeros((height, width, 3), dtype=np.uint8)
img_no_obj[v, u] = aug_points[:, 3:]
img_no_obj = cv2.cvtColor(img_no_obj, cv2.COLOR_RGB2BGR)

#### 

In [28]:
from matplotlib import pyplot as plt
plt.imshow(img_no_obj)
plt.show()


231

#### 绘制点云

In [20]:
from tools.visualize_util import draw_3d_box

vis = o3d.visualization.Visualizer()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(no_object[:,0:3])
pcd.colors = o3d.utility.Vector3dVector(no_object[:,3:] / 256.)

vis.create_window()
vis.add_geometry(pcd)

for obj in obj_list:
    corner = obj.generate_corners3d()
    line_set = draw_3d_box(corner)
    vis.add_geometry(line_set)
    
vis.run()
vis.destroy_window()

#### 

In [4]:
from aug.roiaware_pool3d.roiaware_pool3d_utils import points_in_boxes_cpu


